In [1]:
import os, sys 
sys.path.append(r"F:\pypjts\DSE-REST-API\src")

# Documentation 
https://github.com/innovata/DSE-REST-API  

# [1] 3DEXPERIENCE Platform CLM Agent 생성  
- "F:\pypjts\DSE-REST-API\How to use APIs\credentials"경로에 Agent ID, Password 저장 

# [2] 파이썬 가상환경(Python Virtual Environment) 생성 및 dserestapi 패키지 설치



# [3] 환경셋업

In [2]:
import os, sys 

# <!-- 방법-1 (권장)-->
os.environ["CLM_AGENT_CREDENTIAL_PATH"] = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\CML Agent ID-PW.json"

# <!-- 방법-2 (선택사항) -->
# os.environ["CLM_AGENT_ID"] = "YOUR_ID"
# os.environ["CLM_AGENT_PASSWORD"] = "YOUR_PASSWORD"

os.environ["3DX_PLATFORM_TENANT_URI"] = "https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443"

import dserestapi 

In [3]:
dserestapi

<module 'dserestapi' from 'F:\\pypjts\\DSE-REST-API\\src\\dserestapi\\__init__.py'>

# [4] 사용 예제 | Storages

In [4]:
from dserestapi import Storages
api = Storages()
api 

### [4.1] 스토리지 목록 가져오기 

In [ ]:
# 모든 스토리지 리스트 가져오기
# res = api.get()
res.json()

In [ ]:
!pip install pandas 

In [ ]:
import pandas as pd 
pd.DataFrame(res.json()['cards']).head(5)

In [ ]:
import json 

file = os.path.join(os.getcwd(), "01_Storages", "Storages List.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.2] ObjectStorage 신규 생성 

In [ ]:
# 스토리지 생성
res = api.create(
    stype="ObjectStorage",
    name="TestObject-03.CreatedByRestAPI",
    description="테스트 후 바로 삭제할 것"
)
res 


응답코드--> 409


응답코드가 2xx 또는 3xx 대역이 아닐 경우 아래와 같이 표시됩니다-->
{ '_content': b'{"type":"about:blank","title":"Conflict","status":409,"detai'
              b'l":"Failed to create resource Resource Id TestObject-04.Crea'
              b'tedByRestAPI already exists in workspace with id : dw-global'
              b'-000000-default on the tenant : R1132100527066","instance":"'
              b'/data-factory/api/1/storage"}',
  '_content_consumed': True,
  '_next': None,
  'connection': <requests.adapters.HTTPAdapter object at 0x000001DEFD987380>,
  'cookies': <RequestsCookieJar[]>,
  'elapsed': datetime.timedelta(seconds=8, microseconds=953897),
  'encoding': None,
  'headers': {'content-type': 'application/problem+json', 'transfer-encoding': 'chunked', 'date': 'Fri, 25 Jul 2025 11:24:11 GMT', 'server': 'Microsoft-IIS/7.0'},
  'history': [],
  'raw': <urllib3.response.HTTPResponse object at 0x000001DEFDBAFEE0>,
  'reason': 'Conflict',
  'request': <PreparedRequest [POST]>,
  'status_code': 409,
 

<Response [409]>

In [ ]:
file = os.path.join(os.getcwd(), "01_Storages", "Create A Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.3] 생성한 스토리지 찾기

In [10]:
# 스토리지 검색-1
res = api.search_by_name(
    name="TestObject-04.CreatedByRestAPI", 
    # workspace_id="dw-global-000000-default"
)
res 


응답코드--> 200


<Response [200]>

In [11]:
file = os.path.join(os.getcwd(), "01_Storages", "Search-1 the Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.4] 스토리지 비우기 
--> 해당 기능은 ObjectStorage 에서 다룹니다

### [4.9] 생성한 스토리지 삭제

In [ ]:
with open(file, "r", encoding="utf-8") as f:
    info = json.load(f)
    resource_uuid = info['cards'][0]["resourceUUID"]
resource_uuid

In [ ]:
# 스토리지 검색-2
res = api.search_by_uuid(resource_uuid=resource_uuid)
res 

In [ ]:
res = api.delete(resource_uuid=resource_uuid)
res 

In [ ]:
# 삭제 함수 응답에는 JSON 데이터가 없기 때문에 에러 발생
# res.json()

# [5] 사용 예제 | Object Storage

In [12]:
from dserestapi import Storages
api = Storages()
api 

### [5.1] Object Storage에 파일 업로드 

In [ ]:
# 다시 신규 스토리지 생성
res = api.create(
    stype="ObjectStorage",
    name="TestObject-04.CreatedByRestAPI",
    description="테스트 후 바로 삭제할 것"
)

In [13]:
# 스토리지 검색-1
res = api.search_by_name(
    name="TestObject-04.CreatedByRestAPI", 
)
res 


응답코드--> 200


<Response [200]>

In [14]:
resource_uuid = res.json()['cards'][0]['resourceUUID']
resource_uuid

'1395c39b-eb88-433f-8974-9a3afe9f38a3'

In [ ]:
# 파일 업로드
from dserestapi import ObjectStorage

api = ObjectStorage()
uploading_file = r"E:\__ENGINEER_DRIVE__\__DATASET__\__Dummy Data__\Basic Data Types\all_dtypes.csv"
res = api.upload(resourceUUID=resource_uuid, file=uploading_file, path=None)
res 

In [ ]:
# JSON 데이터 없음
# res.json()

### [5.2] Object Storage에 디렉토리 내의 모든 파일들을 한번에 업로드 

In [ ]:
import os 

top_dir = r"E:\__ENGINEER_DRIVE__\__DATASET__\__Dummy Data__"
uploading_files = []
for root, dirs, files in os.walk(top_dir):
    for file in files:
        uploading_files.append(os.path.join(root, file))

uploading_files

In [ ]:
# 멀티 파일 업로드
import pprint 
pp = pprint.PrettyPrinter(indent=2)

api = ObjectStorage()
res_li = api.upload_files(resourceUUID=resource_uuid, files=uploading_files, path=None)
pp.pprint(res_li)

### [5.3] 스토리지 비우기

In [22]:
api = Storages()
res = api.clear(resource_uuid=resource_uuid)
res 

AttributeError: 'Storages' object has no attribute 'clear'

--> 결과는 플랫폼에서 직접 확인  
res.json() 없음

### [5.4] 스토리지 EXPORT 

In [30]:
from importlib import reload 
import dserestapi
reload(dserestapi)
from dserestapi import Storages

In [31]:
api = Storages()
res = api.export_(resource_uuid=resource_uuid)
res 


응답코드--> 200


<Response [200]>

In [33]:
file = os.path.join(os.getcwd(), "01_Storages", "Export A Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [5.5] 스토리지 IMPORT 

# [6] 사용 예제 | Semantic Graph Index Unit

### [6.1] SGI Storage 신규 생성시 데이터 모델링을 동시에 수행 

In [ ]:
# 데이터 모델링 
data_modeling_config = {
    "datamodel": {
        "classes": [
            {
                "name": "DummyData_01",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            },
            {
                "name": "DummyData_02",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            }
        ]
    }
}

In [ ]:
from dserestapi import Storages
# 스토리지 생성
api = Storages()
res = api.create(
    stype="IndexUnit",
    name="TestSGI_01_CreatedByRestAPI",
    description="SGI 데이터 모델링 테스트 후 삭제할 것",
    config=data_modeling_config
)
res 

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Create A SGI.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [6.2] 데이터 모델 스키마에 맞는 데이터 생성 후 스키마에 맞는 데이터 타입인지 정합성 체크 

In [ ]:
resource_uuid = "b1176f72-a3e2-4afc-9b02-543859e0cdf2"

In [ ]:
# from dserestapi import SemanticGraphIndex
# sgi = SemanticGraphIndex()
# res = sgi.class_count(resourceUUID=resource_uuid, pkg_name="Rawdata", class_name_li=["DummyData_01"])
# with open(os.path.join(os.getcwd(), "Class Count.json"), "w", encoding="utf-8") as f:
#     json.dump(res.json(), f, ensure_ascii=False, indent=2)

In [ ]:
# 데이터 생성 
data = [
    {
        "uri": "Rawdata.DummyData_01.0001",
        "class": "Rawdata.DummyData_01",
        "seq": 1,
        "title": "This is a title",
        "_text": "bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
    {
        "uri": "Rawdata.DummyData_01.0002",
        "class": "Rawdata.DummyData_01",
        "seq": 2,
        "title": "This is a title-2",
        "_text": "(2) bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
]

In [ ]:
# 주입할 데이터의 정합성 체크 
from dserestapi import SemanticGraphIndex
sgi = SemanticGraphIndex()
res = sgi.validateItemsEvent(resourceUUID=resource_uuid, action="AddOrReplaceItem", data=data)
res 

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Validation Items.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [6.3] 생성한 SGI의 타겟 데이터 모델에 데이터를 저장

In [ ]:
res = sgi.ingest(resourceUUID=resource_uuid, data=data)
res 

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Ingest Data.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

플랫폼에서 직접확인

### [6.4] 데이터 주입/적재 결과 알림 받기 

In [ ]:
res = sgi.notification(resourceUUID=resource_uuid)
res 

### [6.5] 저장한 데이터의 object 별 uri 가져오기

In [ ]:
res = sgi.get_uri(resourceUUID=resource_uuid)
res 

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "SGI URIs.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)